In [1]:
import pandas as pd
import numpy as np
import os
import copy
import math

In [2]:
import importlib
import euler_utils
importlib.reload(euler_utils)

<module 'euler_utils' from '/cluster/home/mfrancesc/IVT/eqasim-java/switzerland/WaitTimeValidation/euler_utils.py'>

In [3]:
main_folder = '/cluster/work/ivt_vpl/mfrancesc/SA_scenario/'

In [4]:
last_iter = 100

In [5]:
no_rej_4000 = euler_utils.read_output(main_folder + 'no_rejections_4000', last_iter)

100%|██████████| 101/101 [00:39<00:00,  2.59it/s]


In [6]:
def check_global_avg(global_avg, drt_legs, drt_trips_stats):
    for it, (it_global_avg, it_drt_legs, it_drt_trips_stats) in enumerate(zip(global_avg, drt_legs, drt_trips_stats)):
        print('Iteration ' + str(it))
        print('Global avg: ')
        display(it_global_avg)
        print('Waiting time average from legs: ' + str(it_drt_legs.waitTime.mean()))
        print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))
        print('\n')

In [7]:
check_global_avg(no_rej_4000['global_avg'], no_rej_4000['drt_legs'], no_rej_4000['drt_trips_stats'])

Iteration 0
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Waiting time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global avg: 


<ipython-input-6-0db37c4ee39b>:7: RuntimeWarning: invalid value encountered in double_scalars
  print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,128.889334,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.484093,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

Waiting time average from legs: 345.7571124657932
Delay factor average from legs: 1.7894168038951483


Iteration 99
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,339.516147,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.759257,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Waiting time average from legs: 339.51614709040354
Delay factor average from legs: 1.7592573513395278


Iteration 100
Global avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,396.589044,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.915395,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Waiting time average from legs: 396.58904417155424
Delay factor average from legs: 1.9153947275432603




In [8]:
def check_global_moving_window_avg(global_moving_avg, drt_legs, moving_window, drt_trips_stats):
    for it, it_global_moving_avg in enumerate(global_moving_avg):
        start = 0
        if it >= moving_window:
            start = it - moving_window + 1
        it_drt_legs = pd.concat([drt_legs[j] for j in range(start, it+1)])
        it_drt_trips_stats = pd.concat([drt_trips_stats[j] for j in range(start, it+1)])
        print('Iteration ' + str(it))
        print('Global moving avg: ')
        display(it_global_moving_avg)
        print('Wait time average from legs: ' + str(it_drt_legs.waitTime.mean()))
        print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))

        print('\n')
        

In [9]:
check_global_moving_window_avg(no_rej_4000['global_moving_avg'], no_rej_4000['drt_legs'], 5, no_rej_4000['drt_trips_stats'])

Iteration 0
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global moving avg: 


<ipython-input-8-d7a40f5e209f>:12: RuntimeWarning: invalid value encountered in double_scalars
  print('Delay factor average from legs: ' + str(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum()))


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,128.889334,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.484093,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

Wait time average from legs: 360.8075677718866
Delay factor average from legs: 1.8106161473647004


Iteration 99
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,352.676317,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.793526,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 352.6763167305661
Delay factor average from legs: 1.7935262921082256


Iteration 100
Global moving avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,358.396167,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.810678,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 358.3961666642089
Delay factor average from legs: 1.8106777058234547




In [10]:
def check_global_successive_avg(global_successive_avg, drt_legs, weight, drt_trips_stats):
    iters_wait_time_avg = []
    for it_drt_leg in drt_legs:
        iters_wait_time_avg.append(it_drt_leg.waitTime.mean())
    print(iters_wait_time_avg)
    iters_wait_time_successive_avg = []
    for it in range(len(iters_wait_time_avg)):
        if it == 0:
            iters_wait_time_successive_avg.append(iters_wait_time_avg[it])
            continue
        previous_avg = iters_wait_time_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_wait_time_successive_avg.append(iters_wait_time_avg[it])
            continue
        if np.isnan(iters_wait_time_avg[it]):
            iters_wait_time_successive_avg.append(previous_avg)
            continue
        iters_wait_time_successive_avg.append((1-weight) * previous_avg + weight * iters_wait_time_avg[it])
    
    iters_delay_factor_avg = []
    for it_drt_trips_stats in drt_trips_stats:
        iters_delay_factor_avg.append(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum())
    print(iters_delay_factor_avg)
    iters_delay_factor_successive_avg = []
    for it in range(len(iters_delay_factor_avg)):
        if it == 0:
            iters_delay_factor_successive_avg.append(iters_delay_factor_avg[it])
            continue
        previous_avg = iters_delay_factor_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_delay_factor_successive_avg.append(iters_delay_factor_avg[it])
            continue
        if np.isnan(iters_delay_factor_avg[it]):
            iters_delay_factor_successive_avg.append(previous_avg)
            continue
        iters_delay_factor_successive_avg.append((1-weight) * previous_avg + weight * iters_delay_factor_avg[it])
    
            
    for it, it_global_successive_avg in enumerate(global_successive_avg):
        print('Iteration ' + str(it))
        print('Global successive avg: ')
        display(it_global_successive_avg)
        print('Wait time average from legs: ' + str(iters_wait_time_successive_avg[it]))
        print('Delay factor average from legs: ' + str(iters_delay_factor_successive_avg[it]))
        print('\n')

In [11]:
check_global_successive_avg(no_rej_4000['global_successive_avg'], no_rej_4000['drt_legs'], 0.5, no_rej_4000['drt_trips_stats'])

[nan, 128.88933364728044, 165.6567747131065, 268.49893169310457, 387.34656215282064, 486.91689491134497, 661.3378502121225, 661.2680994256256, 826.6378634604087, 880.9282727302427, 805.5801238410166, 901.8715378079864, 846.6376065297301, 754.8031504527225, 673.7167970834804, 562.3347933205584, 518.1246918476697, 477.01559623415307, 399.4116796058097, 475.4690320911678, 744.0853786913402, 457.7533488485373, 414.49229110370356, 400.92615280878863, 431.90557902905925, 423.1043100063427, 389.2390136096743, 381.91565840818305, 413.6652799621193, 391.1535239978449, 399.5910030826309, 430.9558247239045, 378.8847667650193, 390.51327797548436, 384.1377302423714, 369.896610020525, 375.46959700974287, 430.6227972598189, 388.7512583260496, 412.1150357208829, 431.84248670879725, 373.63890787366614, 365.325761440806, 375.982320680487, 584.8911167815088, 406.9802348277527, 426.9617818829268, 413.3530563014883, 401.4473673707632, 415.91339129565785, 395.02446917986987, 487.55972915288135, 403.62308788

<ipython-input-10-caa4ae87379e>:22: RuntimeWarning: invalid value encountered in double_scalars
  iters_delay_factor_avg.append(np.float64(it_drt_trips_stats.totalTravelTime.sum()) / it_drt_trips_stats.routerUnsharedTime.sum())


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: nan
Delay factor average from legs: nan


Iteration 1
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,128.889334,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.484093,0.0,0.0,0.0,0.0,0.0,0.0,0.0


...

Wait time average from legs: 353.2978662613748
Delay factor average from legs: 1.7997715851177487


Iteration 99
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,346.407007,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.779514,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 346.4070066758892
Delay factor average from legs: 1.7795144682286383


Iteration 100
Global successive avg: 


,stat,avg,median,min,p_5,p_25,p_75,p_95,max
0,waitTime,371.498025,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,delayFactor,1.847455,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Wait time average from legs: 371.49802542372174
Delay factor average from legs: 1.8474545978859493


